In [ ]:
import pandas as pd
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter

In [ ]:
plt.style.use('sensitivity.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
data_short = pd.read_csv('matrix_elements_short.csv', header=None)
NME = np.array(data_short[0])
Iso = np.array(data_short[1])
Names = np.unique(np.array(data_short[1]))
Method = np.array(data_short[2])
Year = np.array([float(x[-4:]) for x in np.array(data_short[3])])

CUPID = np.where(Iso=='100Mo')
nEXO = np.where(Iso=='136Xe')
LEGEND = np.where(Iso=='76Ge')
Cuts = [nEXO, LEGEND, CUPID]

In [ ]:
data_long = pd.read_csv('matrix_elements_all.csv', header=None)
NME_long = np.array(data_long[0])
Iso_long = np.array(data_long[1])
Names_long = np.unique(np.array(data_long[1]))
Method_long = np.array(data_long[2])
Year_long = np.array([float(x[-4:]) for x in np.array(data_long[3])])

CUPID_long = np.where(Iso_long=='100Mo')
nEXO_long = np.where(Iso_long=='136Xe')
LEGEND_long = np.where(Iso_long=='76Ge')
Cuts_long = [nEXO_long, LEGEND_long, CUPID_long]

In [ ]:
Exp = pd.read_csv('comparison.csv')
discover = np.array([7.4e27, 1.3e28, 1.1e27])
disc_tg = discover*np.array(Exp['PhaseSpace'])[:3]*1e-15
sens_tg = np.array(Exp['Sensitivity'])[:3]*np.array(Exp['PhaseSpace'])[:3]*1e-15
Labels = ['nEXO', 'LEGEND-1000', 'CUPID']

In [ ]:
fig, ax = plt.subplots(figsize=(20,5), ncols=5, nrows=1, sharey=True, facecolor='white')

ax[0].set_ylabel('Number of NMEs')
scale = 1
for i,name in enumerate(Names):
  Cut = np.where(Iso==name)
  logbins = np.logspace(np.log10(1),np.log10(100),15)

  ax[i].set_ylim(0,8)
  ax[i].set_xscale('log')
  ax[i].set_xlabel(r'NME $|\mathcal{M}^{0\nu}|^2$')
  h = ax[i].hist(NME[Cut]**2, logbins, histtype='stepfilled', alpha=0.5, color=colors[i], label=r"${{}}^{{{}}} \mathrm{{{}}} $".format(int(name[:-2]), name[-2:]))

  ax[i].vlines(x=np.median(NME[Cut]**2), ymin=0, ymax=8, color=colors[i], ls='--' )
  ax[i].text(x=np.median(NME[Cut]**2)*1.1, y=7.9, s='Median=$({:.1f})^2$'.format(np.median(NME[Cut])), color=colors[i], fontsize=14, rotation=270, va='top')

  kde = st.gaussian_kde(np.log10(NME[Cut]**2), bw_method=0.35)
  nme = np.linspace(1,100,10000)
  ax[i].plot(nme, kde(np.log10(nme))/np.max(kde(np.log10(nme)))*np.max(h[0]), color=colors[i])
  
  ax[i].legend(loc='upper left', fontsize=12)
plt.savefig('nme_hist.pdf')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,5), ncols=5,nrows=1,sharey=True,facecolor='white')

ax[0].set_ylabel(r'Nuclear Matrix Element $\mathcal{M}^{0\nu}$')
for i,name in enumerate(Names):
  Cut = np.where(Iso==name)

  SortedNME = [x for _, x in sorted(zip(Year[Cut], NME[Cut]))]
  ax[i].set_xticklabels(labels=[2005,2010,2015,2020], rotation=45)
  ax[i].set_ylim(0,8)
  ax[i].set_xlabel('Year')
  ax[i].scatter(sorted(Year[Cut]), NME[Cut], color=colors[i], label=r"${{}}^{{{}}} \mathrm{{{}}} $".format(int(name[:-2]), name[-2:]), s=30)
  ax[i].hlines(y=np.median(NME[Cut]), xmin=2005, xmax=2020, color=colors[i], ls='--' )
  ax[i].legend(loc='upper right')

plt.savefig('nme_time.pdf')
plt.show()

In [ ]:
m_e = 511e6 # in meV
g_A = 1.27 
def Getmbb(NME, TG):
  m_bb = m_e**2/(TG * NME**2 * g_A**4)
  return np.sqrt(m_bb)

def GetNME(mbb, TG):
  NME = m_e**2/(TG * mbb**2 * g_A**4)
  return np.sqrt(NME)

In [ ]:
fig, ax1 = plt.subplots()
size = fig.get_size_inches()*fig.dpi


ax1.set_xlim(1e13, 1e15)
ax1.set_ylim(0,50)
ax1.set_xlabel(r'$T_{1/2}^{0\nu} \times \mathrm{G}$', fontsize=20)
ax1.set_ylabel(r'Nuclear Matrix Element $|\mathcal{M}^{0\nu}|^2$', fontsize=20)
ax1.set_xscale('log')
# ax1.set_yscale('log')
ax1.set_title(r'Median Sensitivity at the 90% C.L.', fontweight='bold')
# ax1.tick_params(left=False, which='both')
ax1.yaxis.set_minor_formatter(NullFormatter())

for i,cut in enumerate(Cuts):
  ax1.scatter([sens_tg[i]]*len(NME[cut]), NME[cut]**2, label=Labels[i], color=colors[i], s=25, alpha=1, ec=colors[i])

rot = [290,290,291,310]
for i,x in enumerate([5,10,15,20]):
  NME_x = GetNME(x, np.linspace(1e13,1e15,1000))
  ax1.plot(np.linspace(1e13,1e15,1000), NME_x**2, ls=':', lw=1, color='grey', zorder=0)
  if np.max(NME_x**2) > 50: 
    NME_diff = np.abs(NME_x**2-50)
    xloc = np.linspace(1e13,1e15,1000)[np.where(NME_diff==np.min(NME_diff))]
  else:
    xloc = np.linspace(1e13,1e15,1000)[np.where(NME_x**2==np.max(NME_x**2))]

  ax1.text(x=xloc, 
          y=GetNME(x, xloc)**2, 
          s=r'$\langle m_{{\beta\beta}}\rangle ={}\;$meV'.format(x), 
          rotation=rot[i], 
          fontsize=12, 
          ha='left', 
          va='top', 
          color='grey')

ax1.fill_between(np.linspace(1e13,1e15,1000), GetNME(15, np.linspace(1e13,1e15,1000))**2, color='grey', alpha=0.1)
ax1.text(x=1.5e13, y=1.5, s='Inverted Neutrino Mass Ordering', rotation=0, fontsize=10, color='grey', ma='center', alpha=1)
ax1.legend(loc='upper right', framealpha=1.0, facecolor='white', fontsize=14)

xpos = [-12, -6, -14]

for i,cut in enumerate(Cuts): 
  ax2 = fig.add_axes([ax1.transData.transform((sens_tg[i], 10))[0]/size[0]-7/8*0.05, 0.125, 0.05, 0.755])
  # ax2.set_yscale('log')
  ax2.set_ylim(0,50)
  ax2.set_xlim(-7,1)
  ax2.axis('off')

#   ax2.plot([0]*10, np.linspace(int(np.min(NME[cut]**2)), int(np.max(NME[cut]**2))+2,10, endpoint=True), lw=1.2, color='grey', zorder=0)
#   ax2.plot(np.linspace(-7,0,10, endpoint=True), [int(np.max(NME[cut]**2))+2]*10 ,  lw=1.2, color='grey', zorder=10)
#   ax2.text(x=xpos[i], y=np.max(NME[cut]**2)*1.17, s='# NMEs', fontsize=14, backgroundcolor='white')

#   logbins = np.logspace(np.log10(int(np.min(NME[cut]**2))),np.log10(np.max(NME[cut]**2)+1),9-i)
#   h_nexo, hx_nexo = np.histogram( NME[cut]**2, bins=logbins)
#   ax2.barh(hx_nexo[:-1], -h_nexo, color='None', alpha=1, align='edge', height=np.diff(logbins), ec=colors[i])
#   ax2.barh(hx_nexo[:-1], -h_nexo, color=colors[i], alpha=0.5, align='edge', height=np.diff(logbins))
#   ax2.scatter([0]*len(NME[cut]), NME[cut]**2, label=Labels[i], color=colors[i], s=25, alpha=1, ec=colors[i], zorder=10)
  ax2.scatter([0], np.median(NME[cut]**2), label=Labels[i], color=colors[i], s=30, alpha=1, ec=colors[i], facecolor='white',zorder=10)

plt.savefig('short_hist_sens_with_axes.pdf')
plt.savefig('short_hist_sens_with_axes.png')

In [ ]:
fig, ax1 = plt.subplots()
size = fig.get_size_inches()*fig.dpi

ax1.set_xlim(1e13, 1e15)
ax1.set_ylim(1,100)
ax1.set_xlabel(r'$T_{1/2}^{0\nu} \times \mathrm{G}$')
ax1.set_ylabel(r'Nuclear Matrix Element $|\mathcal{M}^{0\nu}|^2$')
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_title(r'Median Discovery Potential at $3\sigma$', fontweight='bold')

for i,cut in enumerate(Cuts):
  ax1.scatter([disc_tg[i]]*len(NME[cut]), NME[cut]**2, label=Labels[i], color=colors[i], s=25, alpha=1, ec=colors[i])

for i,x in enumerate([5,10,15]):
  NME_x = GetNME(x, np.linspace(1e13,1e15,1000))
  ax1.plot(np.linspace(1e13,1e15,1000), NME_x**2, ls=':', lw=1,color='grey', zorder=0)
  ax1.text(x=3.5e14, 
          y=GetNME(x, 3e14)**2, 
          s=r'$\langle m_{{\beta\beta}}\rangle={}\;$meV'.format(x), 
          rotation=323, 
          fontsize=14, 
          ha='center', 
          va='center', 
          color='grey')

ax1.fill_between(np.linspace(1e13,1e15,1000), GetNME(15, np.linspace(1e13,1e15,1000))**2, color='grey', alpha=0.1)
ax1.text(x=1e13, y=1, s='Inverted Neutrino Mass Ordering \n'+r'$m_{{\beta\beta}}\geq15\;$meV', rotation=323, fontsize=12, color='grey', ma='center')
ax1.legend(loc='upper right', framealpha=1.0, facecolor='white', fontsize=12)

for i,cut in enumerate(Cuts): 
  ax2 = fig.add_axes([ax1.transData.transform((disc_tg[i], 10))[0]/size[0]-7/8*0.05, 0.125, 0.05, 0.755])
  ax2.set_yscale('log')
  ax2.set_ylim(1,100)
  ax2.set_xlim(-7,1)
  ax2.axis('off')
  print(i, (i%3))
  ax2.plot([0]*10, np.linspace(int(np.min(NME[cut]**2)), int(np.max(NME[cut]**2))+2,10, endpoint=True), lw=1.2, color='grey')
  ax2.plot(np.linspace(-7,0,10, endpoint=True), [int(np.max(NME[cut]**2))+2]*10 ,  lw=1.2, color='grey', zorder=0)
  ax2.text(x=-7.5, y=np.max(NME[cut]**2)*1.15-0.02*i, s='# NMEs', fontsize=8, bbox=dict(facecolor='white', edgecolor='None'))

  logbins = np.logspace(np.log10(int(np.min(NME[cut]**2))),np.log10(np.max(NME[cut]**2)+1),9-i)
  h_nexo, hx_nexo = np.histogram( NME[cut]**2, bins=logbins)
  ax2.barh(hx_nexo[:-1], -h_nexo, color='None', alpha=1, align='edge', height=np.diff(logbins), ec=colors[i])
  ax2.barh(hx_nexo[:-1], -h_nexo, color=colors[i], alpha=0.5, align='edge', height=np.diff(logbins))
  ax2.scatter([0]*len(NME[cut]), NME[cut]**2, label=Labels[i], color=colors[i], s=25, alpha=1, ec=colors[i], zorder=10)
  ax2.scatter([0], np.median(NME[cut]**2), label=Labels[i], color=colors[i], s=30, alpha=1, ec=colors[i], facecolor='white',zorder=10)

plt.savefig('short_hist_disc_with_axes.pdf')
# files.download('short_hist_disc_with_axes.pdf')

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=3, sharex=True, sharey=True, facecolor='white')
plt.subplots_adjust(wspace=0, hspace=0)

ymax = 10

title = ['Median Sensitivity \n at the 90% C.L.', 'Median Discovery \n Potential at $3\sigma$']
for i,t_tg in enumerate([sens_tg, disc_tg]): 
  ax[0,i].set_title(title[i], fontsize=16, fontweight='bold')
  ax[2, i].set_xlabel(r'$\langle m_{\beta\beta}\rangle$ [meV]')
  for j,x in enumerate(Cuts): 
    hist, histbins = np.histogram(Getmbb(NME[x], t_tg[j]), bins=np.arange(1,40,2))


    hist_long, histbins_long = np.histogram(Getmbb(NME_long[Cuts_long[j]], t_tg[j]), bins=np.arange(1,40,2))
    # ax[j,i].fill_between(histbins_long[:-1], hist_long, step='post', alpha=0.2, label='All', color=colors[j])

    ax[j,i].fill_between(histbins[:-1], hist, step='post', alpha=0.5, label=Labels[j], color=colors[j])
    ax[j,i].step(histbins[:-1], hist, where='post', color=colors[j], label='')

    # ax[j,i].fill_between(histbins[:-1], hist, step='post', alpha=0.5, label=Labels[j], color=colors[j])
    # ax[j,i].step(histbins[:-1], hist, where='post', color=colors[j])
    ax[j,i].vlines(x=np.median(Getmbb(NME[x], t_tg[j])), ymin=0, ymax=ymax, ls='--', color=colors[j])
    # ax[j,i].vlines(x=np.median(Getmbb(NME_long[Cuts_long[j]], t_tg[j])), ymin=0, ymax=ymax, ls=':', color=colors[j])

    print(np.median(Getmbb(NME[x], t_tg[j])))
    # print(Labels[j])
    # for z in NME[x]:
    #   print(z, Getmbb(z, t_tg[j]), t_tg[j])
    # for k,y in enumerate(Cuts): 
    #   ax[j,i].vlines(x=np.median(Getmbb(NME[y], t_tg[k])), ymin=0, ymax=ymax, ls='--', color=colors[k])
    #   ax[j,i].vlines(x=np.median(Getmbb(NME_long[Cuts_long[k]], t_tg[k])), ymin=0, ymax=ymax, ls=':', color=colors[k])

    ax[j,i].fill_between(x=np.linspace(15,32,10), y1=0, y2=ymax, color='grey', alpha=0.2)

    if j==1:
      ax[j,i].text(16, 3.5, 'Inverted Neutrino \n Mass Ordering', fontsize=12, color='grey', ma='center')

    ax[j,i].set_xlim(0,32)
    ax[j,i].set_ylim(0,ymax)
    if i==0:
      ax[j,i].set_xticks([0,5,10,15,20,25])
      if j==1:
        ax[j,i].set_ylabel('Number of Models')
      ax[j,i].set_yticks(np.arange(0,ymax,4))
    ax[j,i].legend(loc='upper right', fontsize=12)
plt.savefig('short_mbb_6panel_serif.pdf')

In [ ]:
ymax = 7
title = ['Median Sensitivity at the 90% C.L.', 'Median Discovery Potential at $3\sigma$']
tag = ['sens', 'disc']

for i,t_tg in enumerate([sens_tg, disc_tg]): 
  fig, ax = plt.subplots(figsize=(6,10), ncols=1, nrows=3, sharex=True, sharey=False, facecolor='white')
  plt.subplots_adjust(wspace=0.2, hspace=0)
  ax[0].set_title(title[i])
  ax[2].set_xlabel(r'$m_{\beta\beta}$ [meV]')
  for j,x in enumerate([nEXO,LEGEND,CUPID]): 
    hist, histbins = np.histogram(Getmbb(NME[x], t_tg[j]), bins=np.arange(0,40,2))
    
    ax[j].fill_between(histbins[:-1], hist, step='post', alpha=0.5, label=Labels[j], color=colors[j])
    ax[j].step(histbins[:-1], hist, where='post', color=colors[j])
    ax[j].vlines(x=np.median(Getmbb(NME[x], t_tg[j])), ymin=0, ymax=ymax, ls='--', color=colors[j])

    ax[j].fill_between(x=np.linspace(15,32,10), y1=0, y2=ymax, color='grey', alpha=0.2)
    ax[j].text(18, 3, 'Inverted Neutrino \n Mass Ordering', fontsize=14, color='grey', ma='center')

    ax[j].set_xlim(0,32)
    ax[j].set_ylim(0,ymax)
    ax[j].set_ylabel('Counts')
    ax[j].set_yticks(np.arange(0,ymax,1))
    ax[j].legend(loc='upper right', fontsize=14)
  plt.savefig('mbb_3panel_{}.pdf'.format(tag[i]))

In [ ]:
from matplotlib.lines import Line2D

data_short = pd.read_csv('matrix_elements_short.csv', header=None)
Authors = np.array(data_short[3])
Authors_single = np.unique(np.array(data_short[3]))

fig, ax = plt.subplots(figsize=(12,5), ncols=2, nrows=1, sharex=False, sharey=True, facecolor='white')
size = fig.get_size_inches()*fig.dpi

max_range = 35
ax[0].set_ylim(0,max_range)
ax[0].set_xlim(0,max_range)
ax[1].set_xlim(0,max_range)

ax[0].set_ylabel(r'$\langle m_{\beta\beta}\rangle$ for nEXO', fontsize=20)
ax[0].set_xlabel(r'$\langle m_{\beta\beta}\rangle$ for LEGEND-1000', fontsize=20)
ax[1].set_xlabel(r'$\langle m_{\beta\beta}\rangle$ for CUPID', fontsize=20)

for auth in Authors_single:
  cut = np.where(Authors==auth)
  cut_iso = Iso[cut]
  cut_val = NME[cut]
  cut_method = Method[cut][0]
  nexo_val = cut_val[cut_iso=='136Xe']
  legend_val = cut_val[cut_iso=='76Ge']
  cupid_val = cut_val[cut_iso=='100Mo']

  if len(nexo_val)==0 and len(legend_val)==0 and len(cupid_val)==0:
    continue 
  if len(nexo_val)==0:
    nexo_val = [0]
  if len(legend_val)==0:
    legend_val = [0]
  if len(cupid_val)==0:
    cupid_val = [0]
  
  if cut_method == 'IBM':
    marker = 'o'
  if cut_method == 'NSM':
    marker = 'x'
  if cut_method == 'QRPA':
    marker = '^'
  if cut_method == 'EDF':
    marker = 'D'

  nexo_val = Getmbb(nexo_val[0], sens_tg[0])
  legend_val = Getmbb(legend_val[0], sens_tg[1])
  cupid_val = Getmbb(cupid_val[0], sens_tg[2])

  ax[0].scatter(legend_val, nexo_val, color=colors[0], marker=marker, fc='white', ec=colors[0])
  ax[1].scatter(cupid_val, nexo_val, color=colors[1], marker=marker, fc='white', ec=colors[1])


ax[0].plot([0,max_range], [0,max_range], ls=':', color=colors[0])
ax[1].plot([0,max_range], [0,max_range], ls=':', color=colors[1])

ibm = Line2D([0], [0], marker='o', label='IBM', mfc='white', mec=colors[0], ls='None')
nsm = Line2D([0], [0], marker='x', label='NSM', mfc='white', mec=colors[0], ls='None')
qrpa = Line2D([0], [0], marker='^', label='QRPA', mfc='white', mec=colors[0], ls='None')
edf = Line2D([0], [0], marker='D', label='EDF', mfc='white', mec=colors[0], ls='None')
legend_elements = [ibm, nsm, qrpa, edf]
ax[0].legend(handles=legend_elements, loc='upper left', ncol=4, fontsize=11)

ibm = Line2D([0], [0], marker='o', label='IBM', mfc='white', mec=colors[1], ls='None')
qrpa = Line2D([0], [0], marker='^', label='QRPA', mfc='white', mec=colors[1], ls='None')
edf = Line2D([0], [0], marker='D', label='EDF', mfc='white', mec=colors[1], ls='None')
legend_elements = [ibm, qrpa, edf]
ax[1].legend(handles=legend_elements, loc='upper left', ncol=4, fontsize=11)
plt.savefig('mbb_scatter_sens.pdf')

In [ ]:
nexo_mbb = Getmbb(NME[nEXO], disc_tg[0])
legend_mbb = Getmbb(NME[LEGEND], disc_tg[1])
cupid_mbb = Getmbb(NME[CUPID], disc_tg[2])
for l, v in zip(["nEXO", "LEGEND", "CUORE"], [nexo_mbb, legend_mbb, cupid_mbb]):
    print(f"{l}\t", min(v), max(v), len(v))